## Importações

In [1]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, cross_val_score, RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve

## Função para tratamento Outliers

In [2]:
def outliers(data, fator=1.5):
    data_limpa = data.copy()
    
    for col in data_limpa.select_dtypes(include=[np.number]).columns:
        Q1 = data_limpa[col].quantile(0.25)
        Q3 = data_limpa[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - (fator * IQR)
        upper_bound = Q3 + (fator * IQR)
        data_limpa[col] = np.clip(data_limpa[col], lower_bound, upper_bound)
    
    return data_limpa

## Carregamento dos Dados

In [3]:
data_treino = pd.read_csv('/home/caio/github/k-3/data/train.csv')
data_teste = pd.read_csv('/home/caio/github/k-3/data/test.csv')

In [4]:
data_treino_limpo = outliers(data_treino)
data_teste_limpo = outliers(data_teste)

In [5]:
feature_treino = data_treino_limpo.drop(columns=['rainfall']) # Feature Treino
target_treino = data_treino.pop('rainfall')                   # Target Treino
feature_final = data_teste_limpo                              # Feature para Teste final

## Iniciando o Modelo

In [6]:
modelo = CatBoostRegressor(
    iterations=100,
    depth=6,
    learning_rate=0.1,
    loss_function='RMSE',
    verbose=0
)

## Validação Cruzada

In [7]:
dobra = KFold(n_splits=4, shuffle=True, random_state=69)                 # Configurando tipo de dobra                                   
scores = cross_val_score(modelo, feature_treino,
                         target_treino, cv=dobra,
                         scoring='neg_root_mean_squared_error')          # Treinando o modelo sobre validação cruzada
rmse_mean = -np.mean(scores)
print(rmse_mean)

# OBS: RMSE menor é melhor,
#      Métrica kaggle maior é melhor !
# MVP: 0.31684929778141385

0.31684929778141385


## Modelo Final

In [18]:
modelo.fit(feature_treino, target_treino)
previsao = modelo.predict(feature_final)
previsao1= modelo.predict(feature_treino)

## Gerando arquivo CSV

In [9]:
caminho_destino = 'previsoes/cat/Cat_v1.3.csv'
resultado = pd.DataFrame({
    'id': data_teste['id'],
    'rainfall': previsao     
})

resultado.to_csv(caminho_destino, index=False)


In [15]:
caminho_meta = '/home/caio/github/k-3/meta_features/cat_final.csv'
meta_f = pd.DataFrame({
    'catboost': previsao
})

meta_f.to_csv(caminho_meta, index=False)

In [19]:
caminho_meta = '/home/caio/github/k-3/meta_features/cat_treino.csv'
meta_f = pd.DataFrame({
    'catboost': previsao1
})

meta_f.to_csv(caminho_meta, index=False)

In [20]:
previsao.shape

(730,)

In [21]:
previsao1.shape

(2190,)